# Experiments

#### Imports:

In [47]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf


from tensorflow import keras
from sklearn import preprocessing

#### Aux functions:

In [86]:
def split_data(train_x, train_y, training=0.8, validation=0.5):
    train_size = training

    train_cnt = math.floor(train_x.shape[0] * train_size)
    x_train = train_x.iloc[0:train_cnt].values
    y_train = train_y.iloc[0:train_cnt].values
    x_test = train_x.iloc[train_cnt:]
    y_test = train_y.iloc[train_cnt:]

    division = validation

    train_cnt = math.floor(x_test.shape[0] * division)
    x_validate = x_test.iloc[0:train_cnt].values
    y_validate = y_test.iloc[0:train_cnt].values
    x_test = x_test.iloc[train_cnt:].values
    y_test = y_test.iloc[train_cnt:].values

    #Preprocess the data
    #min_max_scaler = preprocessing.MinMaxScaler()
    #x_validate = min_max_scaler.fit_transform(x_validate)
    #x_train = min_max_scaler.fit_transform(x_train)
    #x_test = min_max_scaler.fit_transform(x_test)
    
    return x_train, y_train, x_test, y_test, x_validate, y_validate
    

#### Load data:

In [87]:
frame = pd.read_csv("complete-frame.csv", sep=",")
"""
train_x = pd.concat([frame.no_mutations, frame.line_coverage, frame.isAssertionRoulette, frame.isEagerTest, frame.isLazyTest,
frame.isMysteryGuest, frame.isSensitiveEquality, frame.isResourceOptimism, frame.isForTestersOnly,
frame.isIndirectTesting, frame.LOC_prod, frame.HALSTEAD_prod, frame.RFC_prod, frame.CBO_prod, frame.MPC_prod, frame.IFC_prod, frame.DAC_prod,frame.DAC2_prod, frame.LCOM1_prod, frame.LCOM2_prod,
frame.LCOM3_prod, frame.LCOM4_prod, frame.CONNECTIVITY_prod, frame.LCOM5_prod, frame.COH_prod, frame.TCC_prod,
frame.LCC_prod, frame.ICH_prod, frame.WMC_prod, frame.NOA_prod, frame.NOPA_prod, frame.NOP_prod,
frame.McCABE_prod, frame.BUSWEIMER_prod, frame.LOC_test, frame.HALSTEAD_test, frame.RFC_test, frame.CBO_test,
frame.MPC_test, frame.IFC_test, frame.DAC_test, frame.DAC2_test, frame.LCOM1_test, frame.LCOM2_test,
frame.LCOM3_test, frame.LCOM4_test, frame.CONNECTIVITY_test, frame.LCOM5_test, frame.COH_test, frame.TCC_test,
frame.LCC_test, frame.ICH_test, frame.WMC_test, frame.NOA_test, frame.NOPA_test, frame.NOP_test, frame.McCABE_test,
frame.BUSWEIMER_test, frame.csm_CDSBP, frame.csm_CC, frame.csm_FD, frame.csm_Blob, frame.csm_SC, frame.csm_MC,
frame.csm_LM, frame.csm_FE, frame.prod_readability, frame.test_readability], axis=1).round(2)

"""

data_x = pd.concat([frame.no_mutations,
                     frame.line_coverage,
                     #frame.isEagerTest,
                     frame.LOC_prod, frame.LOC_test, frame.WMC_prod,
                     frame.LCOM1_prod, frame.LCOM2_prod,
                     frame.LCOM4_prod, frame.McCABE_prod,
                     frame.RFC_prod, frame.MPC_prod,
                     frame.RFC_test, frame.MPC_test,
                     frame.LCOM1_test, frame.LCOM2_test,
                     frame.LCOM4_test, frame.LCC_test,
                     frame.LCC_test, frame.WMC_test,
                     frame.McCABE_test, frame.NOP_prod,
                     frame.NOA_prod], axis = 1).round(2)

data_y = pd.concat([frame.mutation], axis = 1)

#labels = [1,2,3,4,5]
#bins = [0,2,4,6,8,10]

#frame['mutation_bins'] = pd.cut(frame.mutation.round(1).mul(10), bins=bins, labels = labels, include_lowest=True)
#frame['mutation_bins'] = pd.cut(frame.mutation, bins=bins, labels = labels, include_lowest=True)
#print(frame.mutation_bins)

#train_y = pd.concat([frame.mutation_bins], axis = 1)

## Experiment #1: Effective Vs. Non Effective
    - 2 classes: effective tests and non-effective tests:
        - Effective > median(mutation_score)
        - Non Effective < media(mutation_score)

In [88]:
#Edit the y axis data according to the description:
labels = [1,2]
bins = [0,frame.mutation.median(),1]
frame['mutation_bins'] = pd.cut(frame.mutation, bins=bins, labels = labels, include_lowest=True)
train_y = pd.concat([frame.mutation_bins], axis = 1)

x_train, y_train, x_test, y_test, x_validate, y_validate = split_data(data_x, data_y)
print(x_train)


# Define the model architecture

model = keras.Sequential()
model.add(keras.layers.Dense(22, input_dim=22, activation=tf.nn.relu))
model.add(keras.layers.Dense(18, activation=tf.nn.relu))
model.add(keras.layers.Dense(12, activation=tf.nn.relu))
model.add(keras.layers.Dense(6, activation=tf.nn.relu))
model.add(keras.layers.Dense(6, activation=tf.nn.softmax))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping_monitor = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)


model.fit(x_train, y_train, epochs=1000, verbose=1,
          validation_data=(x_validate, y_validate),
          callbacks=[early_stopping_monitor])

# Evaluate the model using test dataset.
test_loss, test_acc = model.evaluate(x_test, y_test)

print('Test accuracy:', test_acc)

#Check Overfeet
#test_loss, test_acc = model.evaluate(x_train, y_train)


[[ 36.     0.93  95.   ...   6.     0.     0.  ]
 [ 28.     0.67 271.   ...  30.     0.     0.  ]
 [ 74.     0.97 375.   ...  24.     0.     0.  ]
 ...
 [  4.     1.    95.   ...   8.     0.     2.  ]
 [  4.     1.    95.   ...  21.     0.     2.  ]
 [  6.     1.    87.   ...   7.     0.     1.  ]]
Train on 1928 samples, validate on 241 samples
Epoch 1/1000
1928/1928 [==============================] - 2s 934us/sample - loss: 2.0384 - accuracy: 0.0078 - val_loss: 21.5392 - val_accuracy: 0.0000e+00
Epoch 2/1000
1928/1928 [==============================] - 0s 102us/sample - loss: 0.5565 - accuracy: 0.0099 - val_loss: 5.1906 - val_accuracy: 0.0000e+00
Epoch 3/1000
1928/1928 [==============================] - 0s 95us/sample - loss: 0.2659 - accuracy: 0.0099 - val_loss: 2.0491 - val_accuracy: 0.0041
Epoch 4/1000
1928/1928 [==============================] - 0s 91us/sample - loss: 0.2110 - accuracy: 0.0099 - val_loss: 1.6984 - val_accuracy: 0.0000e+00
Epoch 5/1000
1928/1928 [==================

Test accuracy: 0.049586777


## Experiment #1: Results

| This | is   |
|------|------|
|   a  | table|

## Experiment #2: Predicting Mutation Score
    - 11 categories of mutation score: from 0, 0.1, 0.2, .... 1
    - The goal: predict the mutation score

In [84]:
train_y = pd.concat([frame.mutation], axis = 1).round(1).mul(10)

x_train, y_train, x_test, y_test, x_validate, y_validate = split_data(data_x, data_y)

# Define the model architecture

model = keras.Sequential()
model.add(keras.layers.Dense(22, input_dim=22, activation=tf.nn.relu))
model.add(keras.layers.Dense(18, activation=tf.nn.relu))
model.add(keras.layers.Dense(12, activation=tf.nn.relu))
model.add(keras.layers.Dense(6, activation=tf.nn.relu))
model.add(keras.layers.Dense(11, activation=tf.nn.softmax))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping_monitor = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)


model.fit(x_train, y_train, epochs=1000, verbose=1,
          validation_data=(x_validate, y_validate),
          callbacks=[early_stopping_monitor])

# Evaluate the model using test dataset.
test_loss, test_acc = model.evaluate(x_test, y_test)

print('Test accuracy:', test_acc)

Train on 1928 samples, validate on 241 samples
Epoch 1/1000
1928/1928 [==============================] - 1s 525us/sample - loss: 1.9150 - accuracy: 0.0342 - val_loss: 1.1318 - val_accuracy: 0.0000e+00
Epoch 2/1000
1928/1928 [==============================] - 0s 90us/sample - loss: 0.5137 - accuracy: 0.0099 - val_loss: 1.9083 - val_accuracy: 0.0000e+00
Epoch 3/1000
1928/1928 [==============================] - 0s 90us/sample - loss: 0.2251 - accuracy: 0.0099 - val_loss: 2.0746 - val_accuracy: 0.0000e+00
Epoch 4/1000
1928/1928 [==============================] - 0s 90us/sample - loss: 0.2144 - accuracy: 0.0099 - val_loss: 1.9898 - val_accuracy: 0.0000e+00
Epoch 5/1000
1928/1928 [==============================] - 0s 90us/sample - loss: 0.2043 - accuracy: 0.0099 - val_loss: 1.9171 - val_accuracy: 0.0000e+00
Epoch 6/1000
1928/1928 [==============================] - 0s 97us/sample - loss: 0.1960 - accuracy: 0.0099 - val_loss: 1.9729 - val_accuracy: 0.0000e+00
Epoch 7/1000
1928/1928 [==========

Epoch 54/1000
1928/1928 [==============================] - 0s 102us/sample - loss: 0.1125 - accuracy: 0.0239 - val_loss: 4.2819 - val_accuracy: 0.0000e+00
Epoch 55/1000
1928/1928 [==============================] - 0s 94us/sample - loss: 0.1127 - accuracy: 0.0239 - val_loss: 4.1300 - val_accuracy: 0.0000e+00
Epoch 56/1000
1928/1928 [==============================] - 0s 89us/sample - loss: 0.1095 - accuracy: 0.0280 - val_loss: 4.2660 - val_accuracy: 0.0000e+00
Epoch 57/1000
1928/1928 [==============================] - 0s 93us/sample - loss: 0.1101 - accuracy: 0.0265 - val_loss: 4.3169 - val_accuracy: 0.0000e+00
Epoch 58/1000
1928/1928 [==============================] - 0s 105us/sample - loss: 0.1101 - accuracy: 0.0285 - val_loss: 4.3236 - val_accuracy: 0.0000e+00
Epoch 59/1000
1928/1928 [==============================] - 0s 94us/sample - loss: 0.1101 - accuracy: 0.0233 - val_loss: 4.4014 - val_accuracy: 0.0000e+00
Epoch 60/1000
1928/1928 [==============================] - 0s 95us/sample 

Test accuracy: 0.008264462
